<a href="https://colab.research.google.com/github/Santosh-Gupta/ScientificSummarizationDataSets/blob/master/PublicPython2PointerGeneratorSectionalSummaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a Python 2 notebook, running in Python 3 will not work. 

Make sure you import/copy the sectional summary dataset, the folder name should be 'SemanticScholarAbstractSectionSummaryDataSet' and the path should be 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/'

Highly recommended that you switch the output for the final processed files to a folder in your google drive folder, so you can skip that step the next time.




In [0]:
#Mount Drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import requests
import os
import random
import tarfile
import pandas as pd
import zipfile
import multiprocess
import json
from glob import glob

!apt-get install libmagic-dev
!pip install python-magic
import magic
mime = magic.Magic(mime=True)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,549 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.2 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.2 [68.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.2 [79.5 kB]
Fetched 332 kB in 1s (453 kB/s)
Selecting previously unselected package libmagic-mg

In [0]:
#Needed for pointer-generator

!pip install torch
!pip install pytorch-pretrained-bert
!pip install tensorboardX
!pip install pyrouge
!pip install fastparquet

     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 655kB 8.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex
     |████████████████████████████████| 225kB 2.8MB/s 
     |████████████████████████████████| 61kB 2.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge
     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 61kB 22.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/f3/27/fb/839c776ec8689ff9ee52ad3e91d7d4d848ac6d7545a127d5b0
  Stored in directory: /root/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift


In [0]:
#install Java for stanford tokenizer

!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!java -version

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
#download stanford tokenizer

url = 'http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip'
r = requests.get(url)
with open('stanford-corenlp-full-2018-10-05.zip', 'wb') as f:
    f.write(r.content)

In [0]:
with zipfile.ZipFile('stanford-corenlp-full-2018-10-05.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [0]:
#check jar file

!jar -tvf stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar

#Note, the repo used is for this version of the stanford parser. If changing the version, you need to change this line
# in the repo
# https://github.com/Santosh-Gupta/BertSum/blob/master/src/prepro/data_builder.py#L235

     0 Mon Oct 08 23:22:50 UTC 2018 META-INF/
   176 Mon Oct 08 23:22:50 UTC 2018 META-INF/MANIFEST.MF
 54588 Mon Oct 08 22:56:12 UTC 2018 edu/stanford/nlp/pipeline/StanfordCoreNLP.class
 23392 Mon Oct 08 22:56:26 UTC 2018 edu/stanford/nlp/pipeline/CustomAnnotationSerializer.class
  7847 Mon Oct 08 22:56:28 UTC 2018 edu/stanford/nlp/process/PTBEscapingProcessor.class
  4767 Mon Oct 08 22:56:14 UTC 2018 edu/stanford/nlp/dcoref/Constants.class
   688 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/AliasMatch.class
   760 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/CorefDictionaryMatch.class
 22345 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/DeterministicCorefSieve.class
   534 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/DiscourseMatch.class
   542 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/ExactStringMatch.class
   754 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/Lexi

In [0]:
#Test Stanford Tokenizer 

!echo "Please tokenize this text." | java -cp stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar edu.stanford.nlp.process.PTBTokenizer

Please
tokenize
this
text
.
PTBTokenizer tokenized 5 tokens at 39.22 tokens per second.


In [0]:
#Get forked Bert summarization repos

!git clone -b Python2 https://github.com/Santosh-Gupta/cnn-dailymail.git
# !git clone https://github.com/abisee/pointer-generator.git
!git clone -b fix-flag-parsing https://github.com/f0k/pointer-generator.git

Cloning into 'cnn-dailymail'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 116 (delta 34), reused 0 (delta 0), pack-reused 61
Receiving objects: 100% (116/116), 15.71 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Cloning into 'pointer-generator'...
remote: Enumerating objects: 104, done.
remote: Total 104 (delta 0), reused 0 (delta 0), pack-reused 104
Receiving objects: 100% (104/104), 67.05 KiB | 6.09 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [0]:
if not os.path.exists('Tokenized'):
    os.makedirs('Tokenized')

In [0]:
if not os.path.exists('testDir'):
    os.makedirs('testDir')

In [0]:
from os.path import join as pjoin
thePath = 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/'
filelist = glob(pjoin(thePath, '*'))
print filelist

['gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet01.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet10.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet05.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet02.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet03.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet12.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet17.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet18.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet13.parquet.gzip', 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryData

In [0]:
print filelist

['gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet01.parquet.gzip']


In [0]:
if not os.path.exists('testDir2'):
    os.makedirs('testDir2')

Process files from gdrive into format that can be used by the pointer generator repo

In [0]:
import threading

In [0]:
def thread_function(pfile):
    df = pd.read_parquet(pfile, engine='fastparquet')
    for index, row in df.iterrows():
        sampletext =  row['paperSection'] + ' . \n  \n ' + ' @highlight ' + ' \n  \n . ' + row['Summary']
        sampletext = sampletext.encode('utf-8').strip()
        text_file = open('testDir2/placeholder'+pfile[80:90] + 'z' + str(index)+'.story', "w")
        text_file.write(sampletext)
        text_file.close()

In [0]:
a_threads = []
j = 0
for file in filelist:
    print(file)
    x = threading.Thread(target=thread_function, args=(file,))
    x.start()
    a_threads.append(x)
    if j%5 == 0:
        for thread in a_threads:
            thread.join()
    j=j+1
    
for thread in a_threads:
    thread.join()
    

gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet01.parquet.gzip


Tokenize processed files

In [0]:
os.listdir('testDir2')[0:10]

['placeholdert01.z12374.story',
 'placeholdert01.z338.story',
 'placeholdert01.z10025.story',
 'placeholdert01.z1114.story',
 'placeholdert01.z16366.story',
 'placeholdert01.z30296.story',
 'placeholdert01.z33706.story',
 'placeholdert01.z27220.story',
 'placeholdert01.z14045.story',
 'placeholdert01.z4571.story']

In [0]:
# Preprocess datafor training
!python cnn-dailymail/make_datafiles.py 'testDir2'

Preparing to tokenize testDir2 to cnn_stories_tokenized...
Making list of files to tokenize...
Tokenizing 49224 files in testDir2 and saving in cnn_stories_tokenized...
Untokenizable: ⁎ (U+204E, decimal: 8270)
Untokenizable: ⃝ (U+20DD, decimal: 8413)
Untokenizable: � (U+FFFD, decimal: 65533)
Untokenizable: ⁎ (U+204E, decimal: 8270)
Untokenizable: ‒ (U+2012, decimal: 8210)
Untokenizable: ﹤ (U+FE64, decimal: 65124)
Untokenizable: � (U+FFFD, decimal: 65533)
Untokenizable:  (U+F0B7, decimal: 61623)
Untokenizable: � (U+FFFD, decimal: 65533)
Untokenizable: ‒ (U+2012, decimal: 8210)
Untokenizable: ‒ (U+2012, decimal: 8210)
Untokenizable:  (U+F0B7, decimal: 61623)
Untokenizable: � (U+FFFD, decimal: 65533)
Untokenizable:  (U+F06D, decimal: 61549)
Untokenizable: � (U+FFFD, decimal: 65533)
Untokenizable: ‒ (U+2012, decimal: 8210)
Untokenizable: ‒ (U+2012, decimal: 8210)
Untokenizable:  (U+F0B4, decimal: 61620)
Untokenizable:  (U+F061, decimal: 61537)
Untokenizable: � (U+FFFD, decimal: 65533)

In [0]:
#check files
os.listdir('finished_files/chunked')[0:10]

['val_003.bin',
 'val_001.bin',
 'train_009.bin',
 'train_028.bin',
 'train_022.bin',
 'train_030.bin',
 'train_015.bin',
 'train_032.bin',
 'val_009.bin',
 'train_026.bin']

In [0]:
%cd pointer-generator

/content/pointer-generator


In [0]:
os.listdir('..finished_files/')

Train. Uncomment '%%capture' to suppress output

In [0]:
# %%capture
!python run_summarization.py --mode=train --data_path=../finished_files/chunked/train_* --vocab_path=../finished_files/vocab --log_root=../ --exp_name=trialRun --batch_size=512 --max_enc_steps=400 --max_dec_steps=100 --beam_size=3 --min_dec_steps=10 --coverage=True

W0721 17:07:39.561963 139799651395456 deprecation_wrapper.py:119] From run_summarization.py:324: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0721 17:07:39.562592 139799651395456 deprecation_wrapper.py:119] From run_summarization.py:271: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0721 17:07:39.562779 139799651395456 deprecation_wrapper.py:119] From run_summarization.py:271: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0721 17:07:39.562902 139799651395456 deprecation_wrapper.py:119] From run_summarization.py:272: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0721 17:07:39.562988 139799651395456 run_summarization.py:272] Starting seq2seq_attention in train mode...
max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last wor